## **CHAPTER 3: Looking Inside Large Language Models**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

name_model = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(name_model)

model = AutoModelForCausalLM.from_pretrained(
    name_model,
    device_
)